In [60]:
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

Fix the csv featuring aid received, literacy rates, and GDP per capita. Change year columns into to individual entries for each country. Make series names into column names and values. 

In [61]:
country_aid_gdp_lit_df = pd.read_csv("data/2nd_try_aid_gdp_lit.csv")

In [62]:
#country_aid_gdp_lit_df

In [63]:
value_vars = list(country_aid_gdp_lit_df.columns)[2:]
id_vars = list(country_aid_gdp_lit_df.columns)[:2]
id_vars

['Country Name', 'Series Name']

In [64]:
melted_df = pd.melt(country_aid_gdp_lit_df, id_vars, value_vars)
melted_df.columns = ["Country Name", "Metric", "Year", "Value"]
melted_df.set_index(keys=["Country Name","Year"], inplace=True)
# melted_df.pivot(columns='Metric')
#melted_df

In [65]:
df1 = country_aid_gdp_lit_df.set_index(['Country Name', 'Series Name'])
df1.columns.name = 'year'
df1 = df1.stack().unstack('Series Name')

In [66]:
df1.describe()

Series Name,GDP per capita (current US$),"Literacy rate, adult total (% of people ages 15 and above)",Net official development assistance and official aid received (current US$)
count,1248,1248,1248
unique,1155,183,1221
top,..,..,..
freq,94,1060,22


In [86]:
#The nature of the visualizations I am making means that replacing ".." values 
#is difficult. For example, the size of a bubble is dependent on literacy rates.
#In the data, there is not an entry for literacy rates given each year
#, meaning that I might have to make certain assumptions (like applying 
#the most recent lit rates across the following years with no values). For now,
#we'll leave the ".." as zero. Might fix in excel
df1_temp = df1.replace(to_replace='..', value='0')

In [88]:
df1_temp.head(5)

Series Name       GDP per capita (current US$)  \
Country Name year                                
Afghanistan  1990                            0   
             1991                            0   
             1992                            0   
             1993                            0   
             1994                            0   

Series Name       Literacy rate, adult total (% of people ages 15 and above)  \
Country Name year                                                              
Afghanistan  1990                                                  0           
             1991                                                  0           
             1992                                                  0           
             1993                                                  0           
             1994                                                  0           

Series Name       Net official development assistance and official aid received (current US$)  
Country Name year                                                                              
Afghanistan  1990                                          121720000                           
             1991                                          507530000                           
             1992                                          204900000                           
             1993                                          223800000                           
             1994                                          226420000

In [89]:
#df1.interpolate()

In [68]:
df1.to_csv("clean_data/gdp_lit_aid.csv")

Now inspect aiding spending per year per sector. 

In [69]:
df2 = pd.read_csv("data/aid_spending_by_sector_per_yer.csv")

In [70]:
df2

,Sector(s),Time Period,1990,1991,1992,1993,1994,1995,1996,1997,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,"I.1. Education, Total",NaN,5664.71,4973.52,4793.35,4672.43,5376.85,6054.95,5261.22,4683.23,...,8440.51,9417.38,9329.86,9318.94,9780.09,8793.18,8227.74,9157.74,8050.54,8635.15
1,"I.2. Health, Total",NaN,1613.27,1424.84,1617.43,1654.98,1831.49,2171.42,2283.51,1667.18,...,4447.27,5077.02,4990.22,5108.05,6012.12,6160.02,5938.76,5324.30,6094.47,6393.36
2,"I.5. Government & Civil Society, Total",NaN,1735.06,1387.19,1717.47,1300.48,1425.16,1792.34,1496.31,1130.09,...,11678.39,13221.55,13325.07,13983.96,14531.76,13014.89,12526.83,12952.03,11936.95,11924.34
3,"II. Economic Infrastructure & Services, Total",NaN,8382.07,12182.65,8038.75,9591.39,10805.58,12880.17,11512.02,10127.12,...,12242.26,19522.05,15348.72,19487.56,16998.42,17974.14,20836.01,22160.40,22234.71,22004.57
4,"II.5. Business & Other Services, Total",NaN,394.33,308.94,430.42,346.42,1206.94,659.63,642.43,1008.88,...,1443.38,1631.89,1106.67,1309.79,1746.30,941.00,1245.62,2152.68,1238.58,1385.94
5,"III. Production Sectors, Total",NaN,6878.85,6199.57,8069.91,5780.78,5265.10,5734.47,6503.22,4589.22,...,5766.99,7658.01,6713.06,8179.20,8390.32,7829.39,7595.98,7786.47,7601.32,6104.08
6,"IV. Multi-Sector / Cross-Cutting, Total",NaN,1871.51,1524.03,2041.24,2045.15,1965.18,2627.57,2816.00,3228.01,...,6634.13,6919.81,9207.02,14365.97,11441.33,10466.43,10339.32,10960.91,11776.40,12020.48


In [91]:
df2 = df2.drop(['Time Period'], axis=1)

In [92]:
df2.describe

<bound method NDFrame.describe of                                        Sector(s)     1990      1991     1992  \
0                          I.1. Education, Total  5664.71   4973.52  4793.35   
1                             I.2. Health, Total  1613.27   1424.84  1617.43   
2         I.5. Government & Civil Society, Total  1735.06   1387.19  1717.47   
3  II. Economic Infrastructure & Services, Total  8382.07  12182.65  8038.75   
4         II.5. Business & Other Services, Total   394.33    308.94   430.42   
5                 III. Production Sectors, Total  6878.85   6199.57  8069.91   
6        IV. Multi-Sector / Cross-Cutting, Total  1871.51   1524.03  2041.24   

      1993      1994      1995      1996      1997     1998    ...     \
0  4672.43   5376.85   6054.95   5261.22   4683.23  4492.27    ...      
1  1654.98   1831.49   2171.42   2283.51   1667.18  1593.50    ...      
2  1300.48   1425.16   1792.34   1496.31   1130.09  1946.19    ...      
3  9591.39  10805.58  12880.17  1

In [101]:
df2 = df2.set_index(["Sector(s)"])
df2.head(2)

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Sector(s),,,,,,,,,,,,,,,,,,,,,
"I.1. Education, Total",5664.71,4973.52,4793.35,4672.43,5376.85,6054.95,5261.22,4683.23,4492.27,5038.70,...,8440.51,9417.38,9329.86,9318.94,9780.09,8793.18,8227.74,9157.74,8050.54,8635.15
"I.2. Health, Total",1613.27,1424.84,1617.43,1654.98,1831.49,2171.42,2283.51,1667.18,1593.50,1973.92,...,4447.27,5077.02,4990.22,5108.05,6012.12,6160.02,5938.76,5324.30,6094.47,6393.36


In [186]:
df2.T
df2.index.names = ["Year"]

In [189]:
df2 = df2.T

In [191]:
#it seems all values are here, nothing missing. We'll leave it in this format
#for now as it may be easier to work with in the coming pie chart. 
df2.to_csv("clean_data/pie_data_sector.csv")

Last but not least we'll inspect the data I plan on using for the double axis spending vs life expectancy time series. We also need to create the health spending row.

In [157]:
df3 = pd.read_csv('data/life_expect.csv')
df3

,Country Name,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],...,2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
0,IDA only,53.039285,53.301928,53.582540,53.885894,54.209501,54.558191,54.930012,55.314159,55.710239,...,59.858147,60.459556,61.044044,61.599662,62.116124,62.594243,63.040820,63.459617,63.853122,64.222994
1,Lower middle income,59.513275,59.829442,60.183902,60.514125,60.838366,61.157632,61.483625,61.817486,62.150271,...,64.799075,65.165819,65.529030,65.901517,66.262475,66.596613,66.904789,67.193320,67.458766,67.705178
2,Low & middle income,63.106052,63.326673,63.493558,63.653089,63.868162,64.114060,64.404773,64.718892,65.011587,...,67.538034,67.885040,68.212894,68.555213,68.865134,69.178624,69.459854,69.729661,69.970356,70.193261


In [158]:
df3 = df3.set_index("Country Name")


In [165]:

dd = df2.loc["I.2. Health, Total", :'2015'] 

In [166]:
dd

1990    1613.27
1991    1424.84
1992    1617.43
1993    1654.98
1994    1831.49
1995    2171.42
1996    2283.51
1997    1667.18
1998    1593.50
1999    1973.92
2000    1630.83
2001    1741.14
2002    2474.07
2003    2708.75
2004    3428.49
2005    3538.33
2006    4503.32
2007    4447.27
2008    5077.02
2009    4990.22
2010    5108.05
2011    6012.12
2012    6160.02
2013    5938.76
2014    5324.30
2015    6094.47
Name: I.2. Health, Total, dtype: float64

In [161]:
df3.columns = ["1990", '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998','1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007','2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

In [123]:
df2.keys()

Index(['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype='object')

In [167]:
df3 = df3.append(dd)

In [178]:
df3 = df3.T

Country Name,IDA only,Lower middle income,Low & middle income,"I.2. Health, Total"
1990,53.039285,59.513275,63.106052,1613.27
1991,53.301928,59.829442,63.326673,1424.84
1992,53.582540,60.183902,63.493558,1617.43
1993,53.885894,60.514125,63.653089,1654.98
1994,54.209501,60.838366,63.868162,1831.49
1995,54.558191,61.157632,64.114060,2171.42
1996,54.930012,61.483625,64.404773,2283.51
1997,55.314159,61.817486,64.718892,1667.18
1998,55.710239,62.150271,65.011587,1593.50
1999,56.128856,62.469101,65.281524,1973.92


In [180]:
df3.index.names = ['Years']
df3

Country Name,IDA only,Lower middle income,Low & middle income,"I.2. Health, Total"
Years,,,,
1990,53.039285,59.513275,63.106052,1613.27
1991,53.301928,59.829442,63.326673,1424.84
1992,53.582540,60.183902,63.493558,1617.43
1993,53.885894,60.514125,63.653089,1654.98
1994,54.209501,60.838366,63.868162,1831.49
1995,54.558191,61.157632,64.114060,2171.42
1996,54.930012,61.483625,64.404773,2283.51
1997,55.314159,61.817486,64.718892,1667.18
1998,55.710239,62.150271,65.011587,1593.50


In [193]:
df3.to_csv("clean_data/life_clean.csv")